**Prediction using a combined approach of the custom Pixel CNN model and the HSV thresholding technique**

In [18]:
# import statements
import csv
import os
import cv2
import numpy as np
import time
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model

In [19]:
# preprocessing for the input images
def detect_black_border(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]
    x, y, w, h = cv2.boundingRect(largest_contour)
    shrink_factor = 0.05
    x += int(w * shrink_factor)
    y += int(h * shrink_factor)
    w = int(w * (1 - 2 * shrink_factor))
    h = int(h * (1 - 2 * shrink_factor))
    return x, y, w, h

def img_preprocess(img_path, x, y, w, h, target_size=(224, 224)):
    image = cv2.imread(img_path)
    if x >= 0 and y >= 0 and w > 0 and h > 0:
        bbox = (x, y, x + w, y + h)
        cropped_image = Image.fromarray(image).crop(bbox)
        resized_image = cropped_image.resize(target_size, Image.BILINEAR)
    else:
        resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    return img_to_array(resized_image)

In [20]:
# change this variable to the test dataset path
data_dir="/kaggle/input/ps2test-lc/PS2 Test"  

# change this variable to the path of the saved model uploaded in the models folder as pixelcnn.h5
model=load_model("/kaggle/input/final-model/final-backup-model.h5")

In [9]:
# write the predicted output into the predictions-combined.csv
writer = csv.writer(open("predictions-combined.csv", "w"))
writer.writerow(["sample_name", "label"])

19

In [10]:

# detect smoke function to detect smoke using HSV thresholding techniques
def get_hsv(image):
    blur = cv2.GaussianBlur(image, (21, 21), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 50], dtype="uint8")
    upper = np.array([179, 50, 255], dtype="uint8")
    mask = cv2.inRange(hsv, lower, upper)
    no_smoke_pixels = cv2.countNonZero(mask)

    # Threshold of 12600 pixels has been set
    return no_smoke_pixels > 12600

In [13]:
# getting predictions from both pixel CNN and HSV for each sample

video_folders=os.listdir(data_dir)

start_time = time.time() 

for video in video_folders:
    sample_folders=os.listdir(os.path.join(data_dir, video))
    images=os.listdir(os.path.join(data_dir, video,sample_folders[0]))
    image_path = os.path.join(data_dir, video, sample_folders[0], images[0])
    image_temp = cv2.imread(image_path)
    x1,y1,w,h=detect_black_border(image_temp)
    for sample in os.listdir(os.path.join(data_dir,video)):
        temp=[]
        temp2=[]
        for img_path in os.listdir(os.path.join(data_dir,video,sample)):
            new_img=img_preprocess(os.path.join(data_dir,video,sample,img_path),x1,y1,w,h)
            temp.append(new_img)
            temp2.append(get_hsv(new_img))
        test = np.expand_dims(np.array(temp), axis=0)
        
        prediction_hsv=round(temp2.count(True)/10)
        
        
        if prediction_hsv==1:
            predictions_cnn = model.predict(test)
            predicted_class= 0 if predictions_cnn[0][0] < 0.5 else 1
        else:
            predicted_class=prediction_hsv
            
        sample_name = f"{video}_{sample}"
        with open("predictions-combined.csv", "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([sample_name, predicted_class])
            
            
end_time = time.time() 
runtime_seconds = end_time - start_time
print("Inference Time : ",runtime_seconds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 774ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 762ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 750ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 775ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 793ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 743ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 758ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 758ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 971ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 769ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 746

In [21]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import time

def evaluate_model_on_test_videos(model, data_dir, video_folders_test):
    true_labels = []
    predicted_labels = []
    
    start_time = time.time()
    
    for video in video_folders_test:
        for sample in os.listdir(os.path.join(data_dir, video)):
            true_label = int(sample.split('_')[-1])  
            true_labels.append(true_label)

            temp = []
            temp2 = []
            for img_path in os.listdir(os.path.join(data_dir, video, sample)):
                img_path_full = os.path.join(data_dir, video, sample, img_path)
                x1, y1, w, h = detect_black_border(cv2.imread(img_path_full))
                processed_image = img_preprocess(img_path_full, x1, y1, w, h)
                temp.append(processed_image)
                temp2.append(get_hsv(processed_image))
                
            test_data = np.array(temp)
        
            # Use majority vote from HSV-based prediction for classification
            predictions_hsv = round(temp2.count(True) / 10)
            if predictions_hsv == 1:
                predictions_cnn = model.predict(np.expand_dims(test_data, axis=0))
                predicted_class = 0 if predictions_cnn[0][0] < 0.5 else 1
            else:
                predicted_class = predictions_hsv
                
            predicted_labels.append(predicted_class)
            
    end_time = time.time()
    duration = end_time - start_time

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted')  # Use 'weighted' for multiclass
    recall = recall_score(true_labels, predicted_labels, average='weighted')  # Use 'weighted' for multiclass
    f1 = f1_score(true_labels, predicted_labels, average='weighted')  # Use 'weighted' for multiclass
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    return accuracy, precision, recall, f1, duration, conf_matrix

if __name__ == "__main__":
    data_dir = "/kaggle/input/ps2-lc-pure/PS2Train"
    
    # The remaining 12 videos (20%) of dataset which were kept aside for testing and validation     
    video_test = ['video19', 'video03', 'video39', 'video20', 'video46', 'video10', 'video21', 'video18', 'video09', 'video56', 'video25', 'video15']
    model = load_model("/kaggle/input/final-model/final-backup-model.h5") 

    # Evaluate the model 
    accuracy, precision, recall, f1, duration, conf_matrix = evaluate_model_on_test_videos(model, data_dir, video_test)

    # Print evaluation metrics
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"Duration: {duration} seconds")
    print("Confusion Matrix:")
    print(conf_matrix)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 991ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 784ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 767ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 796ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 777ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 772ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 780ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 740ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 742ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 742ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 918ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 940ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 732ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 764ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 749ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 766ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 